In [15]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/saboa/projects/fine-tune-llama2'

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    
    docs = loader.load()
    
    return docs

In [5]:
extracted_data = load_pdf("data/")

could not convert string to float: '0.00-4199475' : FloatObject (b'0.00-4199475') invalid; use 0.0 instead


In [21]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=20,
        
    )
    
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 281


In [8]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-MiniLM-L6-v2", huggingfacehub_api_token=)
    return embeddings

In [9]:
embeddings = download_huggingface_embeddings()

/home/saboa/projects/fine-tune-llama2/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# test
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [24]:
# load it into Chroma
vectordb = Chroma.from_documents(text_chunks, embeddings)

In [39]:
query = "what metric used"
vectordb.similarity_search(query, k=5)

[Document(page_content='Rand Index (ARI) [ 28], Fowlkes Mallows Score (FMS) [ 21], and Wasserstein Distance (WD) [ 54],\nnormalized to (0,1)range, between predicted and ground-truth labels [88, 70].\nWe similarly evaluate Task 2 (Connections) with three metrics: exact string matching, ROUGE-1', metadata={'page': 2, 'source': 'data/NeurIPS-2023-large-language-models-are-fixated-by-red-herrings-exploring-creative-problem-solving-and-einstellung-effect-using-the-only-connect-wall-dataset-Paper-Datasets_and_Benchmarks.pdf'}),
 Document(page_content='object keys and comments.\n2.2 Tasks and Evaluation Metrics\nThe two dataset tasks: Task 1 (Grouping) , and Task 2 (Connections) are identical to the quiz-show’s\nhuman participant tasks. We evaluate Task 1 (Groupings) via six metrics: number of solved walls,\nnumber of correct groups (max. four per wall), Adjusted Mutual Information (AMI) [ 71], Adjusted\nRand Index (ARI) [ 28], Fowlkes Mallows Score (FMS) [ 21], and Wasserstein Distance (WD) 

In [49]:
llm = LlamaCpp(
    model_path="model/llama-2-7b-chat.Q4_0.gguf",
    model_type="llama",
    max_tokens= 1024,
    temperature= 0
)

/home/saboa/projects/fine-tune-llama2/venv/lib/python3.8/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! model_type is not default parameter.
                model_type was transferred to model_kwargs.
                Please confirm that model_type is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from model/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block

In [50]:
prompt_template = '''Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base. 

{context}

Question: {question}
'''
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=vectordb.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs)

In [51]:
query = "what metric used"
qa_chain.run(query)

ValueError: Requested tokens (562) exceed context window of 512